In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
!python -m spacy download en_core_web_lg

2023-11-25 06:11:24.108040: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 06:11:24.108121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 06:11:24.108163: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 06:11:25.795213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [21]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler
import joblib
scaler = joblib.load('/content/gdrive/MyDrive/DeepLearning/scaler.save')
import spacy
nlp = spacy.load('en_core_web_lg')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words_en = stopwords.words('english')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
new_model = keras.models.load_model('/content/gdrive/MyDrive/DeepLearning/CNN_model.keras')

In [23]:

def vectorize(text):
    texto = text.lower()
    texto = re.sub(r'(@[A-Za-z0-9]+)', '', texto)
    texto = re.sub(r'([^0-9A-Za-z \t])', '', texto)
    texto = re.sub(r'(\w+:\/\/\S+)', '', texto)
    texto = re.sub(r'rt', '', texto)
    texto = word_tokenize(texto)
    texto = [palabra for palabra in texto if palabra not in stop_words_en]
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    texto = nlp(texto).vector
    return texto

In [24]:


def predecir(texto):
  texto = vectorize(texto)
  texto = texto.reshape(1,-1)
  texto = scaler.transform(texto)
  texto = texto.reshape(1,300,1)
  texto = new_model.predict(texto)
  print(f'''
  Category\t Probability
  -----------------------
  Apoyo:\t\t {texto[0][0]}
  Competencia:\t {texto[0][1]}
  Construccion:\t {texto[0][2]}
  Corrupcion: {texto[0][3]}
  Economia:\t {texto[0][4]}
  Exterior:\t\t {texto[0][5]}
  Historia:\t {texto[0][6]}
  Opinion:\t {texto[0][7]}
  Salud: {texto[0][8]}
  Seguridad:\t {texto[0][9]}

  ''')

In [25]:
predecir('Quiero decirles a los conservadores que vayan actuando de manera distinta')

1/1 [==============================] - 7s 7s/step

  Category	 Probability
  -----------------------
  Apoyo:		 0.0025350148789584637
  Competencia:	 0.44774168729782104
  Construccion:	 1.7004931862629746e-07
  Corrupcion: 0.017312975600361824
  Economia:	 2.8336905870673945e-06
  Exterior:		 1.3770174689398118e-07
  Historia:	 3.858171112369746e-05
  Opinion:	 0.5323677659034729
  Salud: 6.710640150231484e-07
  Seguridad:	 1.6137175862240838e-07

  
